<a href="https://colab.research.google.com/github/XplorodoX/EDS_projekt/blob/master/BMI_Uebergewicht.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Einführung in Data Science**

**Fakultät Informatik &ndash; Hochschule Aalen**

**Autoren:** Florian Merlau, Vithusan Ratneswaran, Gianni Balbo

# **Explorative Datenanalyse:**

## **Erste Schritte**

### *Entscheidung welche features wir benötigen*

Wir haben uns für die features ERBMI, ERTPREAT, TEAGE,TESEX, EUDRINK, EUEXERCISE,EUEXFREQ,EUFASTFDFRQ,EUFDSIT,EUGENHTH entschieden, weil wir die Essstörungen im Bezug zum BMI analysieren wollen.

## ***Vorbereitung fürs Coden***



### *Benötigte Bilbliotheken importieren*

Nachinstallieren der Bibliothek:  **iteration-utilities** mit pip

In [ ]:
!pip install iteration-utilities

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from iteration_utilities import duplicates
from iteration_utilities import unique_everseen
import plotly.express as px
import plotly.graph_objects as go
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from google.colab import files
from zipfile import ZipFile
import requests
import math
import seaborn as sns

### Collect/Upload Datenbanken (nur wenn die Laufzeit neu initalisiert wird!)

In [ ]:
def get_Databank():
 uploaded = files.upload()

#get_Databank()

### *Datenbanken importieren*







In [ ]:
atusrost_link = 'https://www.bls.gov/tus/special.requests/atusrost_2016.zip'
atussum_link = 'https://www.bls.gov/tus/special.requests/atussum_2016.zip'

In [ ]:
raw_ehresp = pd.read_csv('ehresp_2016.dat')
raw_atusrost = pd.read_csv('atusrost_2016.dat')
raw_atussum = pd.read_csv('atussum_2016.dat')

## **Analyse der Datenbanken**

### *Auslesen der Tabellen mit Head() zum Inspizieren*

In [ ]:
raw_ehresp.head()

,TUCASEID,TULINENO,EEINCOME1,ERBMI,ERHHCH,ERINCOME,ERSPEMCH,ERTPREAT,ERTSEAT,ETHGT,ETWGT,EUDIETSODA,EUDRINK,EUEAT,EUEXERCISE,EUEXFREQ,EUFASTFD,EUFASTFDFRQ,EUFFYDAY,EUFDSIT,EUFINLWGT,EUSNAP,EUGENHTH,EUGROSHP,EUHGT,EUINCLVL,EUINCOME2,EUMEAT,EUMILK,EUPRPMEL,EUSODA,EUSTORES,EUSTREASON,EUTHERM,EUWGT,EUWIC,EXINCOME1
0,20160101160045,1,2,26.6,3,3,5,40,0,0,0,-1,1,2,2,-1,2,-1,-1,1,2.529098e+07,2,3,1,66,7,1,1,2,1,2,2,4,2,165,-1,0
1,20160101160066,1,2,44.3,3,3,5,30,15,0,0,-1,2,1,1,2,2,-1,-1,1,6.301579e+06,2,4,1,69,7,1,1,2,1,-1,1,2,2,300,-1,0
2,20160101160069,1,2,24.5,3,5,-1,75,0,0,0,-1,2,2,1,1,2,-1,-1,1,9.545378e+06,1,2,1,64,7,2,1,2,1,-1,1,5,1,143,2,0
3,20160101160083,1,2,21.2,1,5,-1,165,0,0,2,-1,1,2,1,3,2,-1,-1,1,3.034797e+06,1,3,1,57,7,2,1,2,1,2,1,1,2,98,2,0
4,20160101160084,1,1,29.7,3,1,3,30,27,0,0,-1,1,1,2,-1,2,-1,-1,1,7.238641e+06,2,2,1,64,7,-1,2,2,3,2,1,1,-1,173,-1,0


In [ ]:
raw_atussum.head()

,TUCASEID,TUFINLWGT,TRYHHCHILD,TEAGE,TESEX,PEEDUCA,PTDTRACE,PEHSPNON,GTMETSTA,TELFS,TEMJOT,TRDPFTPT,TESCHENR,TESCHLVL,TRSPPRES,TESPEMPNOT,TRERNWA,TRCHILDNUM,TRSPFTPT,TEHRUSLT,TUDIARYDAY,TRHOLIDAY,TRTEC,TRTHH,t010101,t010102,t010201,t010299,t010301,t010399,t010401,t010499,t020101,t020102,t020103,t020104,t020199,t020201,t020202,t020203,...,t180704,t180799,t180801,t180802,t180803,t180804,t180805,t180806,t180807,t180899,t180901,t180902,t180903,t180904,t180905,t180999,t181001,t181002,t181099,t181101,t181201,t181202,t181203,t181204,t181205,t181301,t181302,t181399,t181401,t181499,t181501,t181599,t181601,t181801,t189999,t500101,t500103,t500105,t500106,t500107
0,20160101160045,2.458865e+07,-1,62,2,39,1,2,1,5,-1,-1,-1,-1,1,2,-1,0,-1,-1,6,0,-1,0,690,0,25,0,0,0,0,0,75,6,0,0,0,50,0,45,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,20160101160066,5.445941e+06,-1,69,1,37,2,2,2,5,-1,-1,-1,-1,1,2,-1,0,-1,-1,1,0,30,0,600,0,20,0,0,0,0,0,60,0,0,0,0,150,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,20160101160069,8.782622e+06,0,24,2,39,2,2,1,5,-1,-1,2,-1,3,-1,-1,2,-1,-1,1,0,-1,380,940,0,120,0,0,0,0,0,0,0,0,30,0,75,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,60,0,0,0,0
3,20160101160083,3.035910e+06,8,31,2,40,1,2,2,1,2,2,2,-1,3,-1,46944,3,-1,32,1,0,-1,705,635,0,20,0,0,0,0,0,20,50,65,60,0,90,0,50,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,20160101160084,6.978586e+06,-1,59,2,39,1,2,1,1,1,2,-1,-1,1,2,30250,0,-1,12,1,0,-1,0,500,0,80,0,0,0,0,0,30,25,0,0,0,0,0,0,...,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,0,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
raw_atusrost.head()

,TUCASEID,TULINENO,TEAGE,TERRP,TESEX,TXAGE,TXRRP,TXSEX
0,20160101160045,1,62,18,2,0,0,0
1,20160101160045,2,65,20,1,0,0,0
2,20160101160045,3,43,22,1,0,0,0
3,20160101160066,1,69,18,1,0,0,0
4,20160101160066,2,67,20,2,0,0,0


### *Wie viele Werte haben wir für Mann oder Frau in den Datenbanken?*

1.   Listeneintrag
2.   Listeneintrag



In [ ]:
Sum_male = raw_atussum[raw_atussum['TESEX'] == 1 ].value_counts().sum() 
Sum_female = raw_atussum[raw_atussum['TESEX'] == 2].value_counts().sum()
[Sum_female, Sum_male]

[5823, 4670]

### Frauen und Männern leere Einträge gelistet


### *Checken was für Datentypen wir in den Tabellen haben bzw. Gültig sind*

In [ ]:
list(raw_atusrost.dtypes)

[dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64')]

In [ ]:
list(raw_ehresp.dtypes)

[dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64')]

In [ ]:
list(raw_atussum.dtypes)

[dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int6

### *Wie viele insgesamte Einträge wir in allen 3 Datenbanken?*

In [ ]:
x=len(raw_atusrost)
y=len(raw_atussum)
z=len(raw_ehresp)
[x,y,z]

[27884, 10493, 10210]

### *Wie viele leere Einträge haben wir in allen 3 Datenbanken*

Für die Datenbank *Atussum*:

In [ ]:
df_isna_atussum = raw_atussum.isna().sum()
df_isna_atussum = df_isna_atussum.value_counts().sum()
df_isna_atussum

423

Für die Datenbank *Ehresp*:

In [ ]:
df_isna_ehresp= raw_ehresp.isna().sum()
df_isna_ehresp= df_isna_ehresp.value_counts().sum()
df_isna_ehresp

37

Für die Datenbank *Aturost*:

In [ ]:
df_isna_aturost = raw_atusrost.isna().sum()
df_isna_aturost= df_isna_aturost.value_counts().sum()
df_isna_aturost

8

### Nach Duplikate in TUCASEID schauen

In [ ]:
list(unique_everseen(duplicates(raw_atussum['TUCASEID'])))

[]

In [ ]:
list(unique_everseen(duplicates(raw_ehresp['TUCASEID'])))

[]

In [ ]:
list(unique_everseen(duplicates(raw_atusrost['TUCASEID'])))

[20160101160045,
 20160101160066,
 20160101160069,
 20160101160083,
 20160101160084,
 20160101160094,
 20160101160116,
 20160101160135,
 20160101160139,
 20160101160533,
 20160101160546,
 20160101160550,
 20160101160564,
 20160101160569,
 20160101160571,
 20160101160573,
 20160101160574,
 20160101160578,
 20160101160591,
 20160101160602,
 20160101160658,
 20160101160660,
 20160101160663,
 20160101160674,
 20160101160694,
 20160101160734,
 20160101160758,
 20160101160769,
 20160101160813,
 20160101160820,
 20160101160828,
 20160101160835,
 20160101160837,
 20160101160846,
 20160101160862,
 20160101160869,
 20160101160884,
 20160101160894,
 20160101160914,
 20160101160916,
 20160101160917,
 20160101160995,
 20160101161013,
 20160101161019,
 20160101161070,
 20160101161076,
 20160101161080,
 20160101161086,
 20160101161107,
 20160101161115,
 20160101161141,
 20160101161156,
 20160101161189,
 20160101161237,
 20160101161238,
 20160101161245,
 20160101161249,
 20160101161290,
 2016010116131

### *Zusammenführen von den Datenbanken: Atussum & Ehresp in ein neues Dataframe merged_inner*

In [ ]:
merged_inner = pd.merge(raw_atussum,raw_ehresp, on = 'TUCASEID', how ='outer')
merged_inner.head(30)

,TUCASEID,TUFINLWGT,TRYHHCHILD,TEAGE,TESEX,PEEDUCA,PTDTRACE,PEHSPNON,GTMETSTA,TELFS,TEMJOT,TRDPFTPT,TESCHENR,TESCHLVL,TRSPPRES,TESPEMPNOT,TRERNWA,TRCHILDNUM,TRSPFTPT,TEHRUSLT,TUDIARYDAY,TRHOLIDAY,TRTEC,TRTHH,t010101,t010102,t010201,t010299,t010301,t010399,t010401,t010499,t020101,t020102,t020103,t020104,t020199,t020201,t020202,t020203,...,t500103,t500105,t500106,t500107,TULINENO,EEINCOME1,ERBMI,ERHHCH,ERINCOME,ERSPEMCH,ERTPREAT,ERTSEAT,ETHGT,ETWGT,EUDIETSODA,EUDRINK,EUEAT,EUEXERCISE,EUEXFREQ,EUFASTFD,EUFASTFDFRQ,EUFFYDAY,EUFDSIT,EUFINLWGT,EUSNAP,EUGENHTH,EUGROSHP,EUHGT,EUINCLVL,EUINCOME2,EUMEAT,EUMILK,EUPRPMEL,EUSODA,EUSTORES,EUSTREASON,EUTHERM,EUWGT,EUWIC,EXINCOME1
0,20160101160045,2.458865e+07,-1,62,2,39,1,2,1,5,-1,-1,-1,-1,1,2,-1,0,-1,-1,6,0,-1,0,690,0,25,0,0,0,0,0,75,6,0,0,0,50,0,45,...,0,0,0,0,1.0,2.0,26.6,3.0,3.0,5.0,40.0,0.0,0.0,0.0,-1.0,1.0,2.0,2.0,-1.0,2.0,-1.0,-1.0,1.0,2.529098e+07,2.0,3.0,1.0,66.0,7.0,1.0,1.0,2.0,1.0,2.0,2.0,4.0,2.0,165.0,-1.0,0.0
1,20160101160066,5.445941e+06,-1,69,1,37,2,2,2,5,-1,-1,-1,-1,1,2,-1,0,-1,-1,1,0,30,0,600,0,20,0,0,0,0,0,60,0,0,0,0,150,0,0,...,0,0,0,0,1.0,2.0,44.3,3.0,3.0,5.0,30.0,15.0,0.0,0.0,-1.0,2.0,1.0,1.0,2.0,2.0,-1.0,-1.0,1.0,6.301579e+06,2.0,4.0,1.0,69.0,7.0,1.0,1.0,2.0,1.0,-1.0,1.0,2.0,2.0,300.0,-1.0,0.0
2,20160101160069,8.782622e+06,0,24,2,39,2,2,1,5,-1,-1,2,-1,3,-1,-1,2,-1,-1,1,0,-1,380,940,0,120,0,0,0,0,0,0,0,0,30,0,75,0,0,...,0,0,0,0,1.0,2.0,24.5,3.0,5.0,-1.0,75.0,0.0,0.0,0.0,-1.0,2.0,2.0,1.0,1.0,2.0,-1.0,-1.0,1.0,9.545378e+06,1.0,2.0,1.0,64.0,7.0,2.0,1.0,2.0,1.0,-1.0,1.0,5.0,1.0,143.0,2.0,0.0
3,20160101160083,3.035910e+06,8,31,2,40,1,2,2,1,2,2,2,-1,3,-1,46944,3,-1,32,1,0,-1,705,635,0,20,0,0,0,0,0,20,50,65,60,0,90,0,50,...,0,0,0,0,1.0,2.0,21.2,1.0,5.0,-1.0,165.0,0.0,0.0,2.0,-1.0,1.0,2.0,1.0,3.0,2.0,-1.0,-1.0,1.0,3.034797e+06,1.0,3.0,1.0,57.0,7.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,98.0,2.0,0.0
4,20160101160084,6.978586e+06,-1,59,2,39,1,2,1,1,1,2,-1,-1,1,2,30250,0,-1,12,1,0,-1,0,500,0,80,0,0,0,0,0,30,25,0,0,0,0,0,0,...,0,0,0,0,1.0,1.0,29.7,3.0,1.0,3.0,30.0,27.0,0.0,0.0,-1.0,1.0,1.0,2.0,-1.0,2.0,-1.0,-1.0,1.0,7.238641e+06,2.0,2.0,1.0,64.0,7.0,-1.0,2.0,2.0,3.0,2.0,1.0,1.0,-1.0,173.0,-1.0,0.0
5,20160101160094,5.191610e+06,4,16,2,36,3,1,1,5,-1,-1,1,1,3,-1,-1,4,-1,-1,1,0,-1,0,565,0,55,0,0,0,0,0,0,0,0,0,0,90,10,0,...,0,0,0,0,1.0,2.0,30.9,3.0,5.0,-1.0,120.0,9.0,0.0,0.0,2.0,1.0,1.0,1.0,6.0,1.0,1.0,2.0,1.0,5.298292e+06,1.0,2.0,2.0,60.0,7.0,2.0,-1.0,-1.0,2.0,1.0,-1.0,-1.0,-1.0,158.0,1.0,0.0
6,20160101160116,9.391848e+06,5,43,2,43,1,2,1,5,-1,-1,2,-1,1,1,-1,3,1,-1,3,0,-1,120,435,0,10,0,0,0,0,0,0,0,0,0,0,80,0,0,...,0,0,0,0,1.0,1.0,18.3,3.0,1.0,5.0,45.0,0.0,0.0,0.0,-1.0,2.0,2.0,1.0,3.0,2.0,-1.0,-1.0,1.0,9.648727e+06,2.0,1.0,1.0,65.0,7.0,-1.0,1.0,2.0,1.0,-1.0,1.0,2.0,2.0,110.0,2.0,0.0
7,20160101160135,2.023918e+06,0,34,2,39,1,2,1,5,-1,-1,2,-1,3,-1,-1,3,-1,-1,1,0,-1,615,645,0,20,0,0,0,0,0,180,90,0,0,0,5,0,30,...,0,0,0,0,1.0,2.0,40.8,3.0,5.0,-1.0,40.0,0.0,0.0,0.0,1.0,1.0,2.0,1.0,7.0,1.0,1.0,2.0,1.0,2.018001e+06,1.0,3.0,1.0,65.0,7.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,245.0,1.0,0.0
8,20160101160139,9.112257e+06,-1,63,1,46,1,2,1,5,-1,-1,-1,-1,1,1,-1,0,1,-1,1,0,-1,0,510,0,0,0,0,0,0,0,0,0,0,0,0,60,0,0,...,0,0,0,0,1.0,1.0,27.6,3.0,1.0,5.0,60.0,0.0,0.0,0.0,-1.0,2.0,2.0,1.0,7.0,1.0,1.0,2.0,1.0,8.881153e+06,2.0,2.0,2.0,74.0,7.0,-1.0,1.0,2.0,1.0,-1.0,-1.0,-1.0,1.0,215.0,-1.0,0.0
9,20160101160533,2.742447e+06,7,39,2,40,1,2,1,1,2,1,2,-1,1,1,-1,2,1,46,1,0,-1,520,670,0,0,0,0,0,90,0,120,0,0,0,0,40,0,0,...,0,0,0,0,1.0,2.0,28.3,3.0,3.0,5.0,120.0,30.0,0.0,0.0,-1.0,1.0,1.0,1.0,3.0,1.0,5.0,2.0,1.0,2.826905e+06,2.0,3.0,2.0,65.0,7.0,1.0,-1.0,-1.0,2.0,2.0,-1.0,-1.0,-1.0,170.0,2.0,0.0


### Check wie viele leere Einträge wir für ERBMI haben

In [ ]:
merged_inner['ERBMI'].isna().sum()

283

In [ ]:
fig = px.scatter_3d(merged_inner, x="ERBMI", y="t010102",z= 'ERTPREAT')
fig.update_xaxes(), #hide hours outside of 9am-5pm ])
fig.show()

### Zum aufräumen löschen wir die Unnötigen Spalten raus

In [ ]:
merged_inner.head()

,TUCASEID,TUFINLWGT,TRYHHCHILD,TEAGE,TESEX,PEEDUCA,PTDTRACE,PEHSPNON,GTMETSTA,TELFS,TEMJOT,TRDPFTPT,TESCHENR,TESCHLVL,TRSPPRES,TESPEMPNOT,TRERNWA,TRCHILDNUM,TRSPFTPT,TEHRUSLT,TUDIARYDAY,TRHOLIDAY,TRTEC,TRTHH,t010101,t010102,t010201,t010299,t010301,t010399,t010401,t010499,t020101,t020102,t020103,t020104,t020199,t020201,t020202,t020203,...,t500103,t500105,t500106,t500107,TULINENO,EEINCOME1,ERBMI,ERHHCH,ERINCOME,ERSPEMCH,ERTPREAT,ERTSEAT,ETHGT,ETWGT,EUDIETSODA,EUDRINK,EUEAT,EUEXERCISE,EUEXFREQ,EUFASTFD,EUFASTFDFRQ,EUFFYDAY,EUFDSIT,EUFINLWGT,EUSNAP,EUGENHTH,EUGROSHP,EUHGT,EUINCLVL,EUINCOME2,EUMEAT,EUMILK,EUPRPMEL,EUSODA,EUSTORES,EUSTREASON,EUTHERM,EUWGT,EUWIC,EXINCOME1
0,20160101160045,2.458865e+07,-1,62,2,39,1,2,1,5,-1,-1,-1,-1,1,2,-1,0,-1,-1,6,0,-1,0,690,0,25,0,0,0,0,0,75,6,0,0,0,50,0,45,...,0,0,0,0,1.0,2.0,26.6,3.0,3.0,5.0,40.0,0.0,0.0,0.0,-1.0,1.0,2.0,2.0,-1.0,2.0,-1.0,-1.0,1.0,2.529098e+07,2.0,3.0,1.0,66.0,7.0,1.0,1.0,2.0,1.0,2.0,2.0,4.0,2.0,165.0,-1.0,0.0
1,20160101160066,5.445941e+06,-1,69,1,37,2,2,2,5,-1,-1,-1,-1,1,2,-1,0,-1,-1,1,0,30,0,600,0,20,0,0,0,0,0,60,0,0,0,0,150,0,0,...,0,0,0,0,1.0,2.0,44.3,3.0,3.0,5.0,30.0,15.0,0.0,0.0,-1.0,2.0,1.0,1.0,2.0,2.0,-1.0,-1.0,1.0,6.301579e+06,2.0,4.0,1.0,69.0,7.0,1.0,1.0,2.0,1.0,-1.0,1.0,2.0,2.0,300.0,-1.0,0.0
2,20160101160069,8.782622e+06,0,24,2,39,2,2,1,5,-1,-1,2,-1,3,-1,-1,2,-1,-1,1,0,-1,380,940,0,120,0,0,0,0,0,0,0,0,30,0,75,0,0,...,0,0,0,0,1.0,2.0,24.5,3.0,5.0,-1.0,75.0,0.0,0.0,0.0,-1.0,2.0,2.0,1.0,1.0,2.0,-1.0,-1.0,1.0,9.545378e+06,1.0,2.0,1.0,64.0,7.0,2.0,1.0,2.0,1.0,-1.0,1.0,5.0,1.0,143.0,2.0,0.0
3,20160101160083,3.035910e+06,8,31,2,40,1,2,2,1,2,2,2,-1,3,-1,46944,3,-1,32,1,0,-1,705,635,0,20,0,0,0,0,0,20,50,65,60,0,90,0,50,...,0,0,0,0,1.0,2.0,21.2,1.0,5.0,-1.0,165.0,0.0,0.0,2.0,-1.0,1.0,2.0,1.0,3.0,2.0,-1.0,-1.0,1.0,3.034797e+06,1.0,3.0,1.0,57.0,7.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,98.0,2.0,0.0
4,20160101160084,6.978586e+06,-1,59,2,39,1,2,1,1,1,2,-1,-1,1,2,30250,0,-1,12,1,0,-1,0,500,0,80,0,0,0,0,0,30,25,0,0,0,0,0,0,...,0,0,0,0,1.0,1.0,29.7,3.0,1.0,3.0,30.0,27.0,0.0,0.0,-1.0,1.0,1.0,2.0,-1.0,2.0,-1.0,-1.0,1.0,7.238641e+06,2.0,2.0,1.0,64.0,7.0,-1.0,2.0,2.0,3.0,2.0,1.0,1.0,-1.0,173.0,-1.0,0.0


In [ ]:
merged_inner = merged_inner.drop(merged_inner.loc[:, 'TUFINLWGT':'TRYHHCHILD'].columns, axis = 1)
merged_inner = merged_inner.drop(merged_inner.loc[:, 'PEEDUCA':'EEINCOME1'].columns, axis = 1)
merged_inner = merged_inner.drop(merged_inner.loc[:, 'ERHHCH':'ERSPEMCH'].columns, axis = 1)
merged_inner = merged_inner.drop(merged_inner.loc[:, 'ERTSEAT':'EUDIETSODA'].columns, axis = 1)
merged_inner = merged_inner.drop(columns='EUFASTFD')
merged_inner = merged_inner.drop(columns='EUFFYDAY')
merged_inner = merged_inner.drop(columns='EUEAT')
merged_inner = merged_inner.drop(merged_inner.loc[:, 'EUFINLWGT':'EUSNAP'].columns, axis = 1)
merged_inner = merged_inner.drop(merged_inner.loc[:, 'EUGROSHP':].columns, axis = 1)

### Analyse über Min/Max

In [ ]:
def minMax(df,c):
  z= []
  z.append(df[c].max())
  z.append(df[c].min())
  return z

In [ ]:
list_columns = ['ERBMI','ERTPREAT', 'TEAGE', 'EUDRINK', 'EUEXERCISE','EUEXFREQ','EUFASTFDFRQ','EUFDSIT','EUGENHTH']

In [ ]:

print(list((merged_male_EUGENHTH_Excellent_Poor)))['EUGENHTH']


NameError: ignored

In [ ]:
for i in list_columns:
   minMax(merged_male_EUGENHTH_Excellent_Poor,i)

In [ ]:
merged_inner.head()

### Löschen von fehlerhaften Daten

In [ ]:
def unglueltige_Werte_loeschen(df,c):
    for x in df[c]:
        if x < 0: 
            df[c] = df[c].replace([x],np.NaN)

In [ ]:
for i in list_columns:
  unglueltige_Werte_loeschen(merged_inner, i)

list(merged_inner['ERBMI'][::])

In [ ]:
for i in list_columns :
  print(minMax(merged_inner,i))

### ***Spliten zwischen Mann und Frau in zwei neue Datenbanken***

In [ ]:
merged_male = merged_inner[merged_inner['TESEX'] == 1 ]
merged_female = merged_inner[merged_inner['TESEX'] == 2]

In [ ]:
merged_female = merged_female.drop(columns='TESEX')
merged_male = merged_male.drop(columns='TESEX')

In [ ]:
merged_male.describe(include='all').T
for i in list_columns: 
  print(merged_male[i].isna().sum())

In [ ]:
merged_female.describe(include='all').T
for i in list_columns: 
  print(merged_female[i].isna().sum())

## Visuallisieren der Ergebnise

In [ ]:
fig= go.Figure()
fig.add_trace(go.Histogram(x=merged_inner[merged_inner['TESEX'] == 1] ['ERBMI'] , name='Male'))
fig.add_trace(go.Histogram(x=merged_inner[merged_inner['TESEX'] == 2] ['ERBMI'] , name='Female'))
fig.update_traces(opacity = 0.75, xbins=dict(start=-5, end= 65, size=5 ))
fig.show()

In [ ]:
fig = px.scatter(merged_male_EUGENHTH_Excellent_Poor, x='ERBMI', y='EUGENHTH',
                 title="BMI")
fig.show()

## **Bereinigen der Daten, indem ungültige Einträge entfernen oder mit geeigneten Werten**

Female Mean Werte berechnen

In [ ]:
merged_female_mean = merged_female.copy()

for i in list_columns:  
  print(merged_female_mean[i].mean())

print(list_columns)

In [ ]:
for i in list_columns:
  merged_female_mean[i].fillna(merged_female_mean[i].mean(), inplace=True)
  merged_female_mean[i] = merged_female_mean[i].round(2)
  merged_female_mean[i] = merged_female_mean[i].astype(int)
list(merged_female_mean['ERBMI'])

Male Mean Werte berechnen

In [ ]:
merged_male_mean = merged_male.copy()

for i in list_columns:  
  print(merged_male_mean[i].mean())

27.97761327949767
66.35725291657495
48.39207708779443
1.3478739810530953
1.3488372093023255
4.383043922369765
3.125515174222555
1.056870989156893
2.483455474128359


In [ ]:
for i in list_columns:
  merged_male_mean[i].fillna(merged_male_mean[i].mean(), inplace=True)
  merged_male_mean[i] = merged_male_mean[i].round(2)
  merged_male_mean[i] = merged_male_mean[i].astype(int)
list(merged_male_mean['ERBMI'])

#list(merged_male_mean['ERBMI'])

[44,
 27,
 28,
 23,
 39,
 36,
 38,
 20,
 40,
 38,
 30,
 29,
 31,
 27,
 18,
 30,
 27,
 32,
 33,
 22,
 33,
 20,
 23,
 27,
 22,
 21,
 29,
 31,
 24,
 35,
 28,
 21,
 24,
 35,
 27,
 21,
 26,
 30,
 27,
 35,
 27,
 27,
 28,
 23,
 25,
 32,
 34,
 28,
 32,
 28,
 21,
 19,
 28,
 34,
 28,
 26,
 31,
 22,
 34,
 24,
 28,
 34,
 36,
 26,
 27,
 28,
 16,
 28,
 29,
 42,
 29,
 35,
 24,
 21,
 27,
 20,
 39,
 28,
 25,
 24,
 18,
 33,
 31,
 23,
 21,
 25,
 33,
 25,
 23,
 26,
 25,
 24,
 29,
 21,
 27,
 28,
 33,
 20,
 26,
 19,
 30,
 25,
 28,
 25,
 28,
 32,
 34,
 26,
 23,
 31,
 28,
 23,
 37,
 26,
 21,
 25,
 21,
 25,
 29,
 35,
 22,
 23,
 33,
 31,
 36,
 27,
 25,
 25,
 30,
 26,
 25,
 29,
 28,
 26,
 22,
 29,
 27,
 34,
 24,
 30,
 23,
 26,
 31,
 34,
 25,
 25,
 30,
 20,
 25,
 23,
 26,
 36,
 20,
 26,
 28,
 26,
 25,
 27,
 30,
 15,
 32,
 29,
 25,
 26,
 28,
 24,
 26,
 23,
 31,
 22,
 28,
 32,
 27,
 35,
 24,
 31,
 32,
 27,
 27,
 27,
 22,
 27,
 29,
 46,
 27,
 21,
 22,
 35,
 35,
 29,
 24,
 27,
 31,
 23,
 23,
 27,
 33,
 22,
 22,
 30,


In [ ]:
merged_male_mean.head(40)

,TUCASEID,TUFINLWGT,TRYHHCHILD,TEAGE,PEEDUCA,PTDTRACE,PEHSPNON,GTMETSTA,TELFS,TEMJOT,TRDPFTPT,TESCHENR,TESCHLVL,TRSPPRES,TESPEMPNOT,TRERNWA,TRCHILDNUM,TRSPFTPT,TEHRUSLT,TUDIARYDAY,TRHOLIDAY,TRTEC,TRTHH,t010101,t010102,t010201,t010299,t010301,t010399,t010401,t010499,t020101,t020102,t020103,t020104,t020199,t020201,t020202,t020203,t020299,...,t500103,t500105,t500106,t500107,TULINENO,EEINCOME1,ERBMI,ERHHCH,ERINCOME,ERSPEMCH,ERTPREAT,ERTSEAT,ETHGT,ETWGT,EUDIETSODA,EUDRINK,EUEAT,EUEXERCISE,EUEXFREQ,EUFASTFD,EUFASTFDFRQ,EUFFYDAY,EUFDSIT,EUFINLWGT,EUSNAP,EUGENHTH,EUGROSHP,EUHGT,EUINCLVL,EUINCOME2,EUMEAT,EUMILK,EUPRPMEL,EUSODA,EUSTORES,EUSTREASON,EUTHERM,EUWGT,EUWIC,EXINCOME1
1,20160101160066,5.445941e+06,-1,69,37,2,2,2,5,-1,-1,-1,-1,1,2,-1,0,-1,-1,1,0,30,0,600,0,20,0,0,0,0,0,60,0,0,0,0,150,0,0,0,...,0,0,0,0,1.0,2.0,44,3.0,3.0,5.0,30,15.0,0.0,0.0,-1.0,2,1.0,1,2,2.0,3,-1.0,1,6.301579e+06,2.0,4,1.0,69.0,7.0,1.0,1.0,2.0,1.0,-1.0,1.0,2.0,2.0,300.0,-1.0,0.0
8,20160101160139,9.112257e+06,-1,63,46,1,2,1,5,-1,-1,-1,-1,1,1,-1,0,1,-1,1,0,-1,0,510,0,0,0,0,0,0,0,0,0,0,0,0,60,0,0,0,...,0,0,0,0,1.0,1.0,27,3.0,1.0,5.0,60,0.0,0.0,0.0,-1.0,2,2.0,1,7,1.0,1,2.0,1,8.881153e+06,2.0,2,2.0,74.0,7.0,-1.0,1.0,2.0,1.0,-1.0,-1.0,-1.0,1.0,215.0,-1.0,0.0
10,20160101160546,3.165768e+06,1,35,43,1,2,2,1,2,1,2,-1,1,1,153846,2,1,55,1,0,-1,254,778,0,60,0,0,0,0,0,0,0,0,0,0,60,0,30,0,...,0,0,0,0,1.0,1.0,28,3.0,1.0,5.0,120,0.0,0.0,0.0,2.0,1,2.0,2,4,1.0,11,2.0,1,3.315484e+06,2.0,3,2.0,72.0,7.0,-1.0,-1.0,-1.0,2.0,1.0,-1.0,-1.0,-1.0,208.0,2.0,0.0
16,20160101160573,9.150706e+06,16,16,36,1,2,1,5,-1,-1,1,1,3,-1,-1,1,-1,-1,1,0,-1,0,318,0,4,0,0,0,0,0,90,0,0,0,0,0,0,90,0,...,0,0,0,0,1.0,1.0,23,3.0,1.0,-1.0,40,0.0,0.0,0.0,-1.0,1,2.0,1,6,2.0,3,-1.0,1,9.744971e+06,2.0,2,2.0,71.0,7.0,-2.0,-1.0,-1.0,2.0,2.0,-1.0,-1.0,-1.0,170.0,2.0,73.0
17,20160101160574,1.299119e+06,13,70,40,1,1,1,5,-1,-1,-1,-1,1,2,-1,1,-1,-1,1,0,-1,0,360,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,...,0,0,0,0,1.0,1.0,39,1.0,1.0,-1.0,23,0.0,0.0,0.0,-1.0,2,2.0,1,13,2.0,3,-1.0,1,1.458745e+06,2.0,3,2.0,69.0,7.0,-1.0,-1.0,-1.0,2.0,-1.0,-1.0,-1.0,-1.0,267.0,2.0,0.0
18,20160101160578,4.994471e+06,15,68,43,1,2,1,5,-1,-1,-1,-1,1,2,-1,2,-1,-1,1,0,-1,0,480,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1.0,1.0,36,3.0,1.0,5.0,0,0.0,0.0,0.0,1.0,1,2.0,2,4,1.0,1,2.0,1,5.499435e+06,2.0,4,1.0,65.0,7.0,-1.0,2.0,2.0,1.0,1.0,2.0,1.0,-1.0,220.0,2.0,0.0
20,20160101160602,5.529364e+06,17,45,39,1,2,1,1,2,1,2,-1,2,1,61000,1,1,40,1,0,-1,0,570,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1.0,1.0,38,3.0,1.0,5.0,25,0.0,0.0,0.0,2.0,1,2.0,2,4,1.0,3,2.0,1,5.968395e+06,2.0,2,3.0,72.0,7.0,-1.0,1.0,2.0,3.0,1.0,2.0,1.0,2.0,280.0,2.0,0.0
24,20160101160632,4.656346e+06,-1,26,41,1,2,2,1,1,1,2,-1,3,-1,53000,0,-1,-4,1,0,-1,0,415,120,0,0,0,0,0,0,30,30,0,0,0,90,0,60,0,...,0,0,15,0,1.0,1.0,20,3.0,1.0,-1.0,0,242.0,0.0,0.0,-1.0,1,1.0,2,4,2.0,3,-1.0,1,5.074545e+06,2.0,2,1.0,68.0,7.0,-1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,135.0,-1.0,0.0
25,20160101160648,9.007256e+06,-1,63,39,1,2,1,5,-1,-1,-1,-1,3,-1,-1,0,-1,-1,5,0,-1,0,355,0,0,0,0,0,0,0,0,60,0,0,0,65,0,45,0,...,0,0,0,0,1.0,1.0,40,3.0,1.0,-1.0,495,0.0,0.0,0.0,-1.0,1,2.0,1,7,2.0,3,-1.0,1,9.419187e+06,2.0,3,1.0,66.0,7.0,-1.0,1.0,2.0,1.0,2.0,1.0,5.0,2.0,250.0,-1.0,0.0
29,20160101160674,1.533584e+07,-1,65,40,1,2,1,5,-1,-1,-1,-1,1,1,-1,0,2,-1,6,0,-1,0,510,0,0,0,0,0,0,0,0,0,0,0,0,60,0,0,0,...,0,0,0,0,1.0,1.0,38,3.0,1.0,5.0,20,0.0,0.0,0.0,-1.0,2,2.0,1,2,2.0,3,-1.0,1,1.570488e+07,2.0,3,1.0,71.0,7.0,-1.0,1.0,2.0,1.0,-1.0,1.0,3.0,2.0,275.0,-1.0,0.0


## **Alle Prädiktionen die mit dem BMI oder Gewicht verbunden sind, wurden schon vorher gelöscht.**


In [ ]:
merged_male_mean.head()

# **Prädiktion**

#EUEXERCISE TRUE/False

In [ ]:
merged_male_EUEXERCISE_False= merged_male_mean[merged_male_mean['EUEXERCISE']==2]
merged_male_EUEXERCISE_True= merged_male_mean[merged_male_mean['EUEXERCISE']== 1]
merged_female_EUEXERCISE_False= merged_female_mean[merged_female_mean['EUEXERCISE'] == 2]
merged_female_EUEXERCISE_True= merged_female_mean[merged_female_mean['EUEXERCISE'] == 1]

#EUGENHTH in range(1-5) 
# Excellent-Poor (1-5)

In [ ]:
merged_male_EUGENHTH_Excellent= merged_male_mean[merged_male_mean['EUGENHTH'] == 1 ]
merged_male_EUGENHTH_very_good= merged_male_mean[merged_male_mean['EUGENHTH'] ==2 ] 
merged_male_EUGENHTH_good = merged_male_mean[merged_male_mean['EUGENHTH'] == 3 ] 
merged_male_EUGENHTH_fair = merged_male_mean[merged_male_mean['EUGENHTH'] == 4 ]
merged_male_EUGENHTH_poor = merged_male_mean[merged_male_mean['EUGENHTH'] == 5 ]
merged_female_EUGENHTH_Excellent= merged_female_mean[merged_female_mean['EUGENHTH'] == 1 ]
merged_female_EUGENHTH_very_good= merged_female_mean[merged_female_mean['EUGENHTH'] ==2 ]
merged_female_EUGENHTH_good = merged_female_mean[merged_female_mean['EUGENHTH'] == 3 ]
merged_female_EUGENHTH_fair = merged_female_mean[merged_female_mean['EUGENHTH'] == 4 ]
merged_female_EUGENHTH_poor = merged_female_mean[merged_female_mean['EUGENHTH'] == 5 ]
merged_male_EUGENHTH_Excellent_Poor= merged_male_EUGENHTH_Excellent.append(merged_male_EUGENHTH_poor, ignore_index=True, sort=False)
list(merged_male_EUGENHTH_Excellent_Poor)



['TUCASEID',
 'TUFINLWGT',
 'TRYHHCHILD',
 'TEAGE',
 'PEEDUCA',
 'PTDTRACE',
 'PEHSPNON',
 'GTMETSTA',
 'TELFS',
 'TEMJOT',
 'TRDPFTPT',
 'TESCHENR',
 'TESCHLVL',
 'TRSPPRES',
 'TESPEMPNOT',
 'TRERNWA',
 'TRCHILDNUM',
 'TRSPFTPT',
 'TEHRUSLT',
 'TUDIARYDAY',
 'TRHOLIDAY',
 'TRTEC',
 'TRTHH',
 't010101',
 't010102',
 't010201',
 't010299',
 't010301',
 't010399',
 't010401',
 't010499',
 't020101',
 't020102',
 't020103',
 't020104',
 't020199',
 't020201',
 't020202',
 't020203',
 't020299',
 't020301',
 't020302',
 't020303',
 't020399',
 't020401',
 't020402',
 't020499',
 't020501',
 't020502',
 't020601',
 't020602',
 't020699',
 't020701',
 't020799',
 't020801',
 't020899',
 't020901',
 't020902',
 't020903',
 't020904',
 't020905',
 't020999',
 't029999',
 't030101',
 't030102',
 't030103',
 't030104',
 't030105',
 't030106',
 't030108',
 't030109',
 't030110',
 't030111',
 't030112',
 't030199',
 't030201',
 't030202',
 't030203',
 't030204',
 't030299',
 't030301',
 't030302',

#Sport aka EUEXFREQ

In [ ]:
merged_male_EUEXFREQ_true= merged_male_mean[merged_male_mean['EUEXFREQ'] == 1]
merged_male_EUEXFREQ_false= merged_male_mean[merged_male_mean['EUEXFREQ'] == 2]

#Merged-Tabellen für die Korrelation 

In [ ]:

merged_male_EUGENHTH_Excellent_Poor_EuEx= merged_male_EUGENHTH_Excellent_Poor.append(merged_male_EUEXERCISE_True, ignore_index=True, sort=False)

merged_male_EUEX_TRUE_FALSE= merged_male_EUEXERCISE_True.append(merged_male_EUEXERCISE_False, ignore_index=True, sort=False )

merged_male_EUGENHTH_Excellent_Poor_EuEx= merged_male_EUGENHTH_Excellent_Poor.append(merged_male_EUEX_TRUE_FALSE, ignore_index=True, sort=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
corr = merged_male_EUEXFREQ_false.corr()
corr.style.background_gradient(cmap='PuOr')

/usr/local/lib/python3.7/dist-packages/pandas/io/formats/style.py:1126: RuntimeWarning:

All-NaN slice encountered

/usr/local/lib/python3.7/dist-packages/pandas/io/formats/style.py:1127: RuntimeWarning:

All-NaN slice encountered



# EUFDSIT in range(1-3)
# enough_to_eat-often_not_enough_to_eat(1-3)

In [ ]:
merged_male_EUFDSIT_1= merged_male_mean[merged_male_mean['EUFDSIT'] == 1 ]
merged_male_EUFDSIT_2=merged_male_mean[merged_male_mean['EUFDSIT'] == 2 ]
merged_male_EUFDSIT_3=merged_male_mean[merged_male_mean['EUFDSIT'] == 3 ]
merged_female_EUFDSIT_1=merged_female_mean[merged_female_mean['EUFDSIT'] == 1 ]
merged_female_EUFDSIT_2=merged_female_mean[merged_female_mean['EUFDSIT'] == 2 ]
merged_female_EUFDSIT_3=merged_female_mean[merged_female_mean['EUFDSIT'] == 3 ]

#EUDRINK in range(1-2)


In [ ]:
merged_male_EUDRINK_False= merged_male[merged_male['EUDRINK']==2]
merged_male_EUDRINK_True= merged_male[merged_male['EUDRINK']== 1]
merged_female_EUDRINK_False= merged_female[merged_female['EUDRINK'] == 2]
merged_female_EUDRINK_True= merged_female[merged_female['EUDRINK'] == 1]

In [ ]:
 merged_male_EUFDSIT_1['EUFDSIT'].tail()

10482    1
10483    1
10486    1
10489    1
10490    1
Name: EUFDSIT, dtype: int64

In [ ]:
list_columns

['ERBMI',
 'ERTPREAT',
 'TEAGE',
 'EUDRINK',
 'EUEXERCISE',
 'EUEXFREQ',
 'EUFASTFDFRQ',
 'EUFDSIT',
 'EUGENHTH']

In [ ]:
 print(merged_female_mean.isna().sum())

TUCASEID        0
TUFINLWGT       0
TRYHHCHILD      0
TEAGE           0
PEEDUCA         0
             ... 
EUSTREASON    156
EUTHERM       156
EUWGT         156
EUWIC         156
EXINCOME1     156
Length: 458, dtype: int64


In [ ]:
print(merged_male_mean.isna().sum())

TUCASEID        0
TUFINLWGT       0
TRYHHCHILD      0
TEAGE           0
PEEDUCA         0
             ... 
EUSTREASON    127
EUTHERM       127
EUWGT         127
EUWIC         127
EXINCOME1     127
Length: 458, dtype: int64


# Visualisierungen zum Daten-Check

#  Lineare Regression 

---

*   Nearest Neighbor Regression und Support Vector Regression
*   mit nicht-linearen Merkmalen Ihrer Wahl






In [ ]:
fig = px.scatter(merged_male_EUGENHTH_Excellent_Poor, x='ERBMI', y='ERINCOME',
                 title="BMI")

fig.show()

# Klassifikation für BMI> 30


---
  
*   Logistische Regressionmit Nearest Neighbor Classificationund Support Vector Machines 

*   nichtlinearen MerkmalenIhrer Wahl

# Vergleichen Sie ebenfalls andere Regressions und KlassifikationsmodelleIhrer Wahl(z.B. Neuronale Netze)


# Korrelations Matrix

In [1]:
corr = merged_male_EUGENHTH_Excellent_Poor_EuEx.corr()
corr.style.background_gradient(cmap='PuOr')


NameError: ignored